# Weak Supervision

In this notebook, we will discuss the concept of weak supervision in the context of anomaly detection. It will define what is often referred to as an *idealized anomaly detector* and compare it to a fully supervised classifier. The end of the notebook also has some macros for plotting the performance of multiple retrainings in terms of median and 68% confidence intervals.

The usual approach to find a powerful discriminant for distinguishing two classes of data (here signal and background) from each other, is to train a machine learning classifier (e.g. a neural network) to distinguish the classes from each other, based on some input features $x$. At each training iteration, we compare the output of the classifier $f(x)$ to the actual label $y$ (1 for signal, 0 for background) of the training data via the loss function (usually binary cross entropy) and optimize the weights of the classifier such that they match as well as possible. What the classifier learns then is to approximate the likelihood ratio $\frac{p_{sig}}{p_{bkg}}$, which is the most powerful test statistic according to the Neyman Pearson Lemma. This *fully supervised classifier* relies on knowing a-priori which training data are signal and which are background during training.

However, there might be cases where one does not have such truth labels in advance. A prominent case is anomaly detection where one searches for small hints of anomalous signal within an overwhelming background, in a model-agnostic way. Let's imagine the case that we only have our measured data, that consists of background and maybe some signal. In addition, we somehow got an extra sample of just background data. If that was the case, we could just train a classifier to distinguish these two classes (data=sig+bkg vs bkg). Our classifier would approach a likelihood ratio that is monotonically linked to the signal-vs-bkg one $\frac{p_{(sig+bkg)}}{p_{bkg}} = \frac{f_{sig} p_{sig} + (1- f_{sig}) p_{bkg}}{p_{bkg}} = f_{sig} \frac{p_{sig}}{p_{bkg}} + (1 - f_{sig})$ where $f_{sig}$ is the (unknown) signal fraction in the data.

The question is now where to get this magic background-only sample. One might have a very good Monte Carlo simulation, or one might instead generate it in-situ via some data-driven estimate. The latter is the approach that multiple weak supervision methods take, such as [CWoLa Hunting](https://arxiv.org/abs/1902.02634), [SALAD](https://arxiv.org/abs/2212.10579), [CATHODE](https://arxiv.org/abs/2109.00546), [CURTAINS](https://arxiv.org/abs/2203.09470), [FETA](https://arxiv.org/abs/2212.11285). For now, we just assume we already have such a large background-only sample and refer to this idealization as the *idealized anomaly detector* (IAD).

Here, we first illustrate the fully supervised training and will then compare it to the IAD.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import sys

from os.path import exists, join, dirname, realpath
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# adding parent directory to path
parent_dir = dirname(realpath(globals()["_dh"][0]))
sys.path.append(parent_dir)

from sk_cathode.classifier_models.neural_network_classifier import NeuralNetworkClassifier

In [ ]:
# :sunglasses:
plt.style.use('dark_background')

The input data are preprocessed via another script `demos/utils/data_preparation.py`. It downloads the [LHCO R\&D dataset](https://zenodo.org/records/4536377) and applies the preprocessing to extract the conditional feature $m=m_{jj}$ and four auxiliary features $x=(m_{j1}, \Delta m_{jj}, \tau_{21,j1}, \tau_{21,j2})$. Morevoer, it divides the $m$ spectrum into signal region and sidebands, and splits the data into training/validation/test sets. Furthermore, it consists of extra background-only samples, which we will make use of here, as well as an extra signal sample, which we will use in supervised training and for the common test set. By default, there are approximately 0.6% signal events in the "data".

For the purpose of this guide, we will only use the signal region, as the sideband is typically used to estimate the background-only sample in realistic weak supervision methods. Further, we will not use the "conditional feature" $m_{jj}$ for the training, even though this would be possible.

In [ ]:
data_path = "./input_data/"

In [ ]:
# data preparation (download and high-level preprocessing)
if not exists(join(data_path, "innerdata_test.npy")):
    process = subprocess.Popen(f"python {join(parent_dir, 'demos', 'utils', 'data_preparation.py')} --outdir {data_path}", shell=True)
    process.wait()

In [ ]:
# data loading
innerdata_train = np.load(join(data_path, "innerdata_train.npy"))
innerdata_val = np.load(join(data_path, "innerdata_val.npy"))
innerdata_test = np.load(join(data_path, "innerdata_test.npy"))
innerdata_extrabkg_train = np.load(join(data_path, "innerdata_extrabkg_train.npy"))
innerdata_extrabkg_val = np.load(join(data_path, "innerdata_extrabkg_val.npy"))
innerdata_extrabkg_test = np.load(join(data_path, "innerdata_extrabkg_test.npy"))
innerdata_extrasig = np.load(join(data_path, "innerdata_extrasig.npy"))

# splitting up the extra signal into train/val proportionally
# also leaving some for the test set
innerdata_extrasig_test = innerdata_extrasig[:20000]
innerdata_extrasig_train, innerdata_extrasig_val = train_test_split(innerdata_extrasig[20000:],
                                                                    train_size=len(innerdata_train)/(len(innerdata_train)+len(innerdata_val)),
                                                                    random_state=42)

The supervised classifier would realistically be trained on a good simulation of the two classes, where the labels are available. We assume this can be done with plenty of simulated signal, more than we would expect to see in the real data. Thus, we throw all our training data, as well as the extra signal and background into the supervised classifier training set.

In [ ]:
sup_clsf_train_set = np.vstack([innerdata_train,
                            innerdata_extrabkg_train,
                            innerdata_extrasig_train])
sup_clsf_val_set = np.vstack([innerdata_val,
                          innerdata_extrabkg_val,
                          innerdata_extrasig_val])

sup_clsf_train_set = shuffle(sup_clsf_train_set, random_state=42)
sup_clsf_val_set = shuffle(sup_clsf_val_set, random_state=42)

In [ ]:
# either train new NN classifier to distinguish between signal and background

sup_scaler = StandardScaler()
sup_scaler.fit(sup_clsf_train_set[:, 1:-1])

X_train = sup_scaler.transform(sup_clsf_train_set[:, 1:-1])
y_train = sup_clsf_train_set[:, -1]
X_val = sup_scaler.transform(sup_clsf_val_set[:, 1:-1])
y_val = sup_clsf_val_set[:, -1]

sup_classifier_savedir = "./trained_classifiers_supervised_0/"
# Let's protect ourselves from accidentally overwriting a trained model.
if not exists(join(sup_classifier_savedir, "CLSF_models")):
    sup_classifier_model = NeuralNetworkClassifier(save_path=sup_classifier_savedir,
                                                   n_inputs=X_train.shape[1],
                                                   early_stopping=True, epochs=None,
                                                   verbose=True)
    sup_classifier_model.fit(X_train, y_train, X_val, y_val)
else:
    print(f"The model exists already in {sup_classifier_savedir}. Remove first if you want to overwrite.")

In [ ]:
# or alternatively load existing classifer model

sup_scaler = StandardScaler()
sup_scaler.fit(sup_clsf_train_set[:, 1:-1])

sup_classifier_savedir = "./trained_classifiers_supervised_0/"
sup_classifier_model = NeuralNetworkClassifier(save_path=sup_classifier_savedir,
                                               n_inputs=sup_clsf_train_set[:, 1:-1].shape[1],
                                               load=True)

We can now evaluate the performance of our supervised classifier in terms of ROC curves. Another useful metric in anomaly detection is the significance improvement characteristic: how much significance ($\frac{S}{\sqrt(B)}$) we would achieve after applying a cut on our classifier output, divided by the significance without any selection. The efficiency in the x-axis quantifies how tight we apply a cut.

In [ ]:
# now let's evaluate the signal extraction performance

clsf_test_set = np.vstack([innerdata_test,
                           innerdata_extrabkg_test,
                           innerdata_extrasig_test])

X_test = sup_scaler.transform(clsf_test_set[:, 1:-1])
y_test = clsf_test_set[:, -1]

preds_test = sup_classifier_model.predict(X_test)

with np.errstate(divide='ignore', invalid='ignore'):
    fpr, tpr, _ = roc_curve(y_test, preds_test)
    bkg_rej = 1 / fpr
    sic = tpr / np.sqrt(fpr)

    random_tpr = np.linspace(0, 1, 300)
    random_bkg_rej = 1 / random_tpr
    random_sic = random_tpr / np.sqrt(random_tpr)

# ROC curve
plt.plot(tpr, bkg_rej, label="supervised")
plt.plot(random_tpr, random_bkg_rej, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Background Rejection")
plt.yscale("log")
plt.legend(loc="upper right")
plt.show()

# SIC curve
plt.plot(tpr, sic, label="supervised")
plt.plot(random_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()

This classifier clearly yields good separation between signal and background data.

Now we would like to move on to training an idealized anomaly detector. We just take the "data" with mostly background and a small fraction of signal, and train a classifier to distinguish it from a pure background sample.

In [ ]:
# assigning label 1 to "data"
iad_clsf_train_data = innerdata_train.copy()
iad_clsf_train_data[:, -1] = np.ones_like(iad_clsf_train_data[:, -1])
iad_clsf_val_data = innerdata_val.copy()
iad_clsf_val_data[:, -1] = np.ones_like(iad_clsf_val_data[:, -1])

# and label 0 to background
iad_clsf_train_bkg = innerdata_extrabkg_train.copy()
iad_clsf_train_bkg[:, -1] = np.zeros_like(iad_clsf_train_bkg[:, -1])
iad_clsf_val_bkg = innerdata_extrabkg_val.copy()
iad_clsf_val_bkg[:, -1] = np.zeros_like(iad_clsf_val_bkg[:, -1])

# mixing together and shuffling
iad_clsf_train_set = np.vstack([iad_clsf_train_data, iad_clsf_train_bkg])
iad_clsf_val_set = np.vstack([iad_clsf_val_data, iad_clsf_val_bkg])
iad_clsf_train_set = shuffle(iad_clsf_train_set, random_state=42)
iad_clsf_val_set = shuffle(iad_clsf_val_set, random_state=42)

In [ ]:
# either train new NN classifier to distinguish between "data" and background

iad_scaler = StandardScaler()
iad_scaler.fit(iad_clsf_train_set[:, 1:-1])

X_train = iad_scaler.transform(iad_clsf_train_set[:, 1:-1])
y_train = iad_clsf_train_set[:, -1]
X_val = iad_scaler.transform(iad_clsf_val_set[:, 1:-1])
y_val = iad_clsf_val_set[:, -1]

iad_classifier_savedir = "./trained_classifiers_idealized-ad_0/"
# Let's protect ourselves from accidentally overwriting a trained model.
if not exists(join(iad_classifier_savedir, "CLSF_models")):
    iad_classifier_model = NeuralNetworkClassifier(save_path=iad_classifier_savedir,
                                                   n_inputs=X_train.shape[1],
                                                   early_stopping=True, epochs=None,
                                                   verbose=True)
    iad_classifier_model.fit(X_train, y_train, X_val, y_val)
else:
    print(f"The model exists already in {iad_classifier_savedir}. Remove first if you want to overwrite.")

In [ ]:
# or alternatively load existing classifer model

iad_scaler = StandardScaler()
iad_scaler.fit(iad_clsf_train_set[:, 1:-1])

iad_classifier_savedir = "./trained_classifiers_idealized-ad_0/"
iad_classifier_model = NeuralNetworkClassifier(save_path=iad_classifier_savedir,
                                               n_inputs=iad_clsf_train_set[:, 1:-1].shape[1],
                                               load=True)

In [ ]:
# now let's evaluate the signal extraction performance on the same test set

clsf_test_set = np.vstack([innerdata_test,
                           innerdata_extrabkg_test,
                           innerdata_extrasig_test])

X_test = iad_scaler.transform(clsf_test_set[:, 1:-1])
y_test = clsf_test_set[:, -1]

preds_test = iad_classifier_model.predict(X_test)

with np.errstate(divide='ignore', invalid='ignore'):
    fpr, tpr, _ = roc_curve(y_test, preds_test)
    bkg_rej = 1 / fpr
    sic = tpr / np.sqrt(fpr)

    random_tpr = np.linspace(0, 1, 300)
    random_bkg_rej = 1 / random_tpr
    random_sic = random_tpr / np.sqrt(random_tpr)

# ROC curve
plt.plot(tpr, bkg_rej, label="idealized AD")
plt.plot(random_tpr, random_bkg_rej, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Background Rejection")
plt.yscale("log")
plt.legend(loc="upper right")
plt.show()

# SIC curve
plt.plot(tpr, sic, label="idealized AD")
plt.plot(random_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()

The plots above should also show quite good significance improvement, however a bit lower than in the fully supervised case. Under perfect training conditions, the two should be equal, but in practice we would need more and more training data with decreasing signal fractions to achieve this.

For the fun of it, let's compare the two approaches more thoroughly in terms of their signal extraction performance, rather than looking at two individual trainings. We will train ten classifiers each with the same data, then show their performance in terms of median and 68% confidence interval bands.

In [ ]:
# either train new NN classifiers to distinguish between signal and background

sup_scaler = StandardScaler()
sup_scaler.fit(sup_clsf_train_set[:, 1:-1])

X_train = sup_scaler.transform(sup_clsf_train_set[:, 1:-1])
y_train = sup_clsf_train_set[:, -1]
X_val = sup_scaler.transform(sup_clsf_val_set[:, 1:-1])
y_val = sup_clsf_val_set[:, -1]

sup_model_list = []
for i in range(10):
    _classifier_savedir = f"./trained_classifiers_supervised_{i}/"     
    sup_model_list.append(NeuralNetworkClassifier(save_path=_classifier_savedir,
                                                  n_inputs=X_train.shape[1],
                                                  early_stopping=True, epochs=None,
                                                  verbose=True))

    # We don't want to overwrite the model if it already exists.
    if not exists(join(_classifier_savedir, "CLSF_models")):
        sup_model_list[-1].fit(X_train, y_train, X_val, y_val)
    else:
        print(f"The model exists already in {_classifier_savedir}. Remove first if you want to overwrite. Loading its best state now.")
        sup_model_list[-1].load_best_model()

In [ ]:
# or alternatively load existing classifer models

sup_scaler = StandardScaler()
sup_scaler.fit(sup_clsf_train_set[:, 1:-1])

sup_model_list = []
for i in range(10):
    _classifier_savedir = f"./trained_classifiers_supervised_{i}/"     
    sup_model_list.append(NeuralNetworkClassifier(save_path=_classifier_savedir,
                                                  n_inputs=sup_clsf_train_set[:, 1:-1].shape[1],
                                                  load=True))

In [ ]:
# the same goes for the IAD: either train from scratch

iad_scaler = StandardScaler()
iad_scaler.fit(iad_clsf_train_set[:, 1:-1])

X_train = iad_scaler.transform(iad_clsf_train_set[:, 1:-1])
y_train = iad_clsf_train_set[:, -1]
X_val = iad_scaler.transform(iad_clsf_val_set[:, 1:-1])
y_val = iad_clsf_val_set[:, -1]

iad_model_list = []
for i in range(10):
    _classifier_savedir = f"./trained_classifiers_idealized-ad_{i}/"     
    iad_model_list.append(NeuralNetworkClassifier(save_path=_classifier_savedir,
                                                  n_inputs=X_train.shape[1],
                                                  early_stopping=True, epochs=None,
                                                  verbose=True))

    # We don't want to overwrite the model if it already exists.
    if not exists(join(_classifier_savedir, "CLSF_models")):
        iad_model_list[-1].fit(X_train, y_train, X_val, y_val)
    else:
        print(f"The model exists already in {_classifier_savedir}. Remove first if you want to overwrite. Loading its best state now.")
        iad_model_list[-1].load_best_model()

In [ ]:
# or alternatively load existing IAD models

iad_scaler = StandardScaler()
iad_scaler.fit(iad_clsf_train_set[:, 1:-1])

iad_model_list = []
for i in range(10):
    _classifier_savedir = f"./trained_classifiers_idealized-ad_{i}/"     
    iad_model_list.append(NeuralNetworkClassifier(save_path=_classifier_savedir,
                                                  n_inputs=iad_clsf_train_set[:, 1:-1].shape[1],
                                                  load=True))

In [ ]:
# now let's evaluate all models and compute their median ROC and SIC, as well as 68% bands

clsf_test_set = np.vstack([innerdata_test,
                           innerdata_extrabkg_test,
                           innerdata_extrasig_test])

# we will infeer all ROC and SIC values onto a common TPR grid
common_tpr = np.linspace(0, 1, 300)

X_test_sup = sup_scaler.transform(clsf_test_set[:, 1:-1])
y_test_sup = clsf_test_set[:, -1]

# first supervised classifiers
sup_bkg_rejs = []
sup_sics = []
for model in sup_model_list:
    preds_test = model.predict(X_test_sup)
    with np.errstate(divide='ignore', invalid='ignore'):
        fpr, tpr, _ = roc_curve(y_test, preds_test)
        sup_bkg_rejs.append(np.interp(common_tpr, tpr, 1/fpr))
        sup_sics.append(np.interp(common_tpr, tpr, tpr / np.sqrt(fpr)))

# then the same with the IAD
X_test_iad = iad_scaler.transform(clsf_test_set[:, 1:-1])
y_test_iad = clsf_test_set[:, -1]

iad_bkg_rejs = []
iad_sics = []
for model in iad_model_list:
    preds_test = model.predict(X_test_iad)
    with np.errstate(divide='ignore', invalid='ignore'):
        fpr, tpr, _ = roc_curve(y_test, preds_test)
        iad_bkg_rejs.append(np.interp(common_tpr, tpr, 1/fpr))
        iad_sics.append(np.interp(common_tpr, tpr, tpr / np.sqrt(fpr)))

with np.errstate(divide='ignore', invalid='ignore'):
    random_bkg_rej = 1 / common_tpr
    random_sic = common_tpr / np.sqrt(common_tpr)

# ROC curves
plt.plot(common_tpr,
         np.median(np.stack(sup_bkg_rejs, axis=0), axis=0),
         label="supervised")
plt.fill_between(common_tpr,
                 np.quantile(np.stack(sup_bkg_rejs, axis=0), q=.16, axis=0),
                 np.quantile(np.stack(sup_bkg_rejs, axis=0), q=.84, axis=0),
                 color=plt.gca().lines[-1].get_color(),
                 alpha=0.3)
plt.plot(common_tpr,
         np.median(np.stack(iad_bkg_rejs, axis=0), axis=0),
         label="idealized AD")
plt.fill_between(common_tpr,
                 np.quantile(np.stack(iad_bkg_rejs, axis=0), q=.16, axis=0),
                 np.quantile(np.stack(iad_bkg_rejs, axis=0), q=.84, axis=0),
                 color=plt.gca().lines[-1].get_color(),
                 alpha=0.3)
plt.plot(common_tpr, random_bkg_rej, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Background Rejection")
plt.yscale("log")
plt.legend(loc="upper right")
plt.show()

# SIC curves
plt.plot(common_tpr,
         np.median(np.stack(sup_sics, axis=0), axis=0),
         label="supervised")
plt.fill_between(common_tpr,
                 np.quantile(np.stack(sup_sics, axis=0), q=.16, axis=0),
                 np.quantile(np.stack(sup_sics, axis=0), q=.84, axis=0),
                 color=plt.gca().lines[-1].get_color(),
                 alpha=0.3)
plt.plot(common_tpr,
         np.median(np.stack(iad_sics, axis=0), axis=0),
         label="idealized AD")
plt.fill_between(common_tpr,
                 np.quantile(np.stack(iad_sics, axis=0), q=.16, axis=0),
                 np.quantile(np.stack(iad_sics, axis=0), q=.84, axis=0),
                 color=plt.gca().lines[-1].get_color(),
                 alpha=0.3)
plt.plot(common_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()

We see that the IAD is not as good as the supervised classifier here. For that we would probably need much more training data. Also, one sees that the supervised classifier has almost negligible variance in performance, whereas the IAD varies from run to run more. It makes sense as there is only a small number of signal events in the training data.

However, the IAD still performs pretty well and comes increasingly close to a fully supervised model at tight cuts. This is remarkable, as there was no explicit signal information provided to the classifier during the training.